In [8]:
import yaml
from pymongo import MongoClient
import time
from datetime import datetime,timedelta
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/karinapatel/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
client = MongoClient()
db = client['capstone']
coll=db['twitter']

In [3]:
def load_api_key(filename='twitter_api_key.yaml'):
    """Load Yelp API client ID and client secret and return them as a dictionary."""
    with open(filename) as f:
        return yaml.load(f)

In [4]:
from twitter import *
key = load_api_key()
consumer_key = key['consumer_key']
consumer_secret = key['consumer_secret']
access_token = key['access_token']
access_token_secret = key['access_token_secret']

t = Twitter(
    auth=OAuth(access_token, access_token_secret,consumer_key, consumer_secret))

In [119]:
x=t.search.tweets(q=("kroger"),lang='en', count=100,tweet_mode='extended',geocode="37.781157,-122.398720,500mi")['statuses']

In [146]:
x=t.search.tweets(q=("jewel food"),lang='en', count=100,tweet_mode='extended',geocode="41.878485,-87.627893,100mi")['statuses']

In [147]:
len(x)

2

In [14]:
c = 0
lst = []
s = []
sent=[]
sid=SentimentIntensityAnalyzer()
for i,tweet in enumerate(x):
            if tweet['id'] not in s:
                date = tweet['created_at']
                current = datetime.strptime(date, '%a %b %d %H:%M:%S +0000 %Y')
                lst.append(tweet['full_text'])
                s.append(tweet['id'])
                ss = sid.polarity_scores(tweet['full_text'])
                sent.append(ss)
                #coll.insert_one({'tweet': tweet, 'sentiment':ss})

In [15]:
lst

[]

In [11]:
len(x)

20

In [12]:
sid=SentimentIntensityAnalyzer()
ss = sid.polarity_scores("I like candy yum")
ss

{'neg': 0.0, 'neu': 0.444, 'pos': 0.556, 'compound': 0.3612}

In [276]:
coll=db['grocery_stores']
words = ['safeway','albertsons','ralphs','kroger','trader joe\'s','whole foods','jewel-osco', 'pavilions', 'food 4 less','randalls','tom thumb','star market','vons','united supermarkets','acme markets', 'carrs']
req_count = 0
for query in words:
    print("starting: ",query)
    current = datetime.now()
    Prior = datetime.now() - timedelta(days=3)
    c = 0
    lst = []
    s = []
    q = query + " -filter:retweets"
    while current>Prior:
        if c == 0:
            #Query = t.search.tweets(q=q,lang='en', count=100,tweet_mode='extended',geocode="41.878485,-87.627893,100mi")['statuses']
            Query = t.search.tweets(q=q,result_type='recent',count=100,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
        else:
            Query = t.search.tweets(q=q,result_type='recent',count=100,max_id=max_id,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
        for i,tweet in enumerate(Query['statuses']):
            if tweet['id'] not in s and query in tweet['full_text'].lower():
                date = tweet['created_at']
                current = datetime.strptime(date, '%a %b %d %H:%M:%S +0000 %Y')
                lst.append(tweet['full_text'])
                s.append(tweet['id'])
                ss = sid.polarity_scores(tweet['full_text'])
                coll.insert_one({'full_data': tweet,'text': tweet['full_text'],'keyword':query, 'sentiment':ss})
        req_count+=1
        max_id = tweet['id']
        if req_count>178:
            print("sleep time")
            time.sleep(60*15)
            req_count = 0
print("finished pull")

starting:  safeway
starting:  albertsons
starting:  ralphs
starting:  kroger
starting:  trader joe's
starting:  whole foods
starting:  jewel-osco
starting:  pavilions
starting:  food 4 less
starting:  randalls
starting:  tom thumb
starting:  star market
starting:  vons
starting:  united supermarkets
starting:  acme markets
starting:  carrs
finished pull


In [273]:
req_count

9

In [256]:
#time.sleep(20*60)

coll=db['ff_stores']

fast_foods = ["mcdonald's",'starbucks','subway',"wendy's",'burger king','taco bell',"dunkin donuts",'chick fil-a','pizza hut',"domino's",'panera bread','sonic',
 'kfc','chipotle',"carl's jr",'dairy queen',"arby's",'little caesars','jack in the box','popeyes louisiana kitchen',"papa john's",'panda express',
 'whataburger',"jimmy john's","zaxby's",'five guys',"culver's","bojangles'",'steak n shake','wingstop',"papa murphy's","checkers","jersey mike's",
 'qdoba mexican',"church's chicken",'el pollo loco','del taco','white castle','tim hortons',"moe's southwest grill",'firehouse subs','boston market',
 "jason's deli",'in-n-out','baskin-robbins',"mcalister's deli","auntie anne's","captain d's",'jamba juice']

#req_count = 0
for query in fast_foods:
    print("starting: ",query)
    current = datetime.now()
    Prior = datetime.now() - timedelta(days=2)
    c = 0
    lst = []
    s = []
    q = query + " -filter:retweets"
    while current>Prior:
        if c == 0:
            Query = t.search.tweets(q=q,result_type='recent',count=100,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
        else:
            Query = t.search.tweets(q=q,result_type='recent',count=100,max_id=max_id,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
        for i,tweet in enumerate(Query['statuses']):
            if tweet['id'] not in s and query in tweet['full_text'].lower():
                date = tweet['created_at']
                current = datetime.strptime(date, '%a %b %d %H:%M:%S +0000 %Y')
                lst.append(tweet['full_text'])
                s.append(tweet['id'])
                ss = sid.polarity_scores(tweet['full_text'])
                coll.insert_one({'full_data': tweet,'text': tweet['full_text'],'keyword':query, 'sentiment':ss})
        req_count+=1
        max_id = tweet['id']
        if req_count>178:
            print("sleep time")
            time.sleep(60*15)
            req_count = 0
print("finished pull")


starting:  mcdonald's
sleep time
starting:  starbucks
sleep time
starting:  subway
sleep time
starting:  wendy's
sleep time
starting:  burger king
starting:  taco bell
starting:  dunkin donuts
starting:  chick fil-a
starting:  pizza hut
sleep time
sleep time
sleep time
sleep time
starting:  domino's
starting:  panera bread
sleep time
starting:  sonic
sleep time
starting:  kfc
sleep time
starting:  chipotle
starting:  carl's jr
starting:  dairy queen
starting:  arby's
starting:  little caesars
starting:  jack in the box
starting:  popeyes louisiana kitchen
starting:  papa john's
starting:  panda express
starting:  whataburger
starting:  jimmy john's
sleep time
starting:  zaxby's
starting:  five guys
starting:  culver's
starting:  bojangles'
starting:  steak n shake
starting:  wingstop
starting:  papa murphy's
starting:  checkers
starting:  jersey mike's
starting:  qdoba mexican
starting:  church's chicken
starting:  el pollo loco
starting:  del taco
starting:  white castle
starting:  ti

KeyError: 'next_results'

In [139]:
#t.oauth2.token(access_token = '1099364574652915712-n2qrqEOXBeRYpdILcrmbdsK5hLjET8')
t.application.rate_limit_status()

{'rate_limit_context': {'access_token': '1099364574652915712-n2qrqEOXBeRYpdILcrmbdsK5hLjET8'},
 'resources': {'lists': {'/lists/list': {'limit': 15,
    'remaining': 15,
    'reset': 1553720295},
   '/lists/memberships': {'limit': 75, 'remaining': 75, 'reset': 1553720295},
   '/lists/subscribers/show': {'limit': 15,
    'remaining': 15,
    'reset': 1553720295},
   '/lists/members': {'limit': 900, 'remaining': 900, 'reset': 1553720295},
   '/lists/subscriptions': {'limit': 15, 'remaining': 15, 'reset': 1553720295},
   '/lists/show': {'limit': 75, 'remaining': 75, 'reset': 1553720295},
   '/lists/ownerships': {'limit': 15, 'remaining': 15, 'reset': 1553720295},
   '/lists/subscribers': {'limit': 180, 'remaining': 180, 'reset': 1553720295},
   '/lists/members/show': {'limit': 15, 'remaining': 15, 'reset': 1553720295},
   '/lists/statuses': {'limit': 900, 'remaining': 900, 'reset': 1553720295}},
  'application': {'/application/rate_limit_status': {'limit': 180,
    'remaining': 179,
    '

In [249]:
fast_foods = ["mcdonald's",'starbucks','subway',"wendy's",'burger king','taco bell',"dunkin donuts",'chick fil-a','pizza hut',"domino's",'panera bread','sonic',
 'kfc','chipotle',"carl's jr",'dairy queen',"arby's",'little caesars','jack in the box','popeyes louisiana kitchen',"papa john's",'panda express',
 'whataburger',"jimmy john's","zaxby's",'five guys',"culver's","bojangles'",'steak n shake','wingstop',"papa murphy's","checkers","jersey mike's",
 'qdoba mexican',"church's chicken",'el pollo loco','del taco','white castle','tim hortons',"moe's southwest grill",'firehouse subs','boston market',
 "jason's deli",'in-n-out','baskin-robbins',"mcalister's deli",'noodles & company',"auntie anne's","captain d's",'jamba juice']



In [269]:
list(coll.find().limit(5))[0]

{'_id': ObjectId('5c9b30f7aaedff5572f7cfb2'),
 'full_data': {'created_at': 'Wed Mar 27 08:14:28 +0000 2019',
  'id': 1110817347403689984,
  'id_str': '1110817347403689984',
  'full_text': "I've never eaten at McDonald's my favourite games console is\n\nMobile",
  'truncated': False,
  'display_text_range': [0, 68],
  'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 1084877911541379072,
   'id_str': '1084877911541379072',
   'name': 'Ted Cubiss',
   'screen_name': 'TCubiss',
   'location': '',
   'description': 'Sub2Pewdiepie',
   'url': None,
   'entities': {'description': {'urls': []}},
   'protected': Fals